In [3]:
from datasets import load_dataset

dataset = load_dataset('Kili/plastic_in_river', num_proc=6)

print(dataset)


/Users/carter/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/carter/.pyenv/versions/3.11.4/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for Kili/plastic_in_river contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Kili/plastic_in_river
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'litter'],
        num_rows: 3407
    })
    test: Dataset({
        features: ['image', 'litter'],
        num_rows: 427
    })
    validation: Dataset({
        features: ['image', 'litter'],
        num_rows: 425
    })
})


In [12]:
import os
from tqdm import tqdm

# only creating datasets for train and validation not test
os.makedirs('datasets/images/train', exist_ok=True)
os.makedirs('datasets/images/test', exist_ok=True)
os.makedirs('datasets/images/validation', exist_ok=True)

CLEAN = 0
OIL_SPILL = 1 # not used here
TRASH = 2

class_labels = [CLEAN, OIL_SPILL, TRASH]

def create_dataset(data, split):
    data = data[split]
    data_size = len(data)

    # create class sub dirs
    for label in class_labels:
        os.makedirs(f'datasets/images/{split}/{label}', exist_ok=True)

    for idx, sample in enumerate(tqdm(data, total=data_size, desc=f"Sorting data in split {split}")):
        image = sample['image']
        labels = sample['litter']['label']
        # targets = []
        
        if(len(labels) > 0):
            class_id = TRASH
        else:
            class_id = CLEAN

        # saving image as png to directory
        image.save(f'datasets/images/{split}/{class_id}/{idx}.png')


splits = ["train", "test", "validation"]

for split in splits:
    create_dataset(dataset, split)

Sorting data in split validation: 100%|██████████| 425/425 [01:50<00:00,  3.84it/s]


In [27]:
import os
import glob
import random
import shutil

START_DIR = "oil/"
TARGET_WORD = "oil"

matching_directories = [os.path.join(dirpath, dirname) for dirpath, dirnames, _ in os.walk(START_DIR) for dirname in dirnames if TARGET_WORD in dirname]

img_ordering = []

for dir in matching_directories:
  # go through all images in subdirs
  imgs = [file for file in glob.iglob(dir + '/*.jpg')]
  for img in imgs:
    img_ordering.append(img) # create total ordering of imgs

# shuffle
random.shuffle(img_ordering)

img_count = len(img_ordering)

train_data = img_ordering[:int((img_count+1)*.60)]
val_data = img_ordering[int((img_count+1)*.60):int((img_count+1)*.80)]
test_data = img_ordering[int((img_count+1)*.80):]

datas = [(train_data, "train"), (val_data, "validation"), (test_data, "test")]
sorted_img_count = 0
for data, label in datas:
  os.makedirs(f'datasets/images/{label}/1', exist_ok=True)
  idx = 0
  for img in data:
    shutil.copy(img, f"datasets/images/{label}/1/{idx}.jpg")
    sorted_img_count += 1
    idx += 1

print(f"Sorted {img_count} images\t {img_count} == {sorted_img_count}")


Sorted 256 images	 256 == 256
